En este nuevo lab, vamos hacer una introduccion a la libreria de Dask. Dask es una liberia de codigo abierto y desarollada en cordinacion con Numpy Pandas y Scikit-Learn. Basicamente nos permite paralelizar procesos de python, y incluso hacer clusters de maquinas para nuestros procesos. Las posibilidades son ilimitadas, pero en este caso nos centraremos en la carga de grande dataframes, y como tratarlos posteriormente en pandas  
La gracia de utilizar dask, es el utilizar los lazy dataframes, que es como una forma de trabajar con los datos "por encima" , y hasta que realmente no los necesitamos, no los procesamos.  
  
    
  
    
Una vez explicado esto, empezamos!

In [ ]:
# %load_ext autotime ## Utilizamos este complemento para poder monitorizar los tiempos que tarda en realizar cada accion
import dask.dataframe as dd # importamos de dask el dataframe
from dask.diagnostics import ProgressBar # y la barra de progreso, para saber "donde estamos"

Ahora vamos a cargar todos los archivos de una carpeta, dentro de nuestra variable df, lo creara en un lazy dataframe, por eso todo funciona de forma tan rapida.

In [2]:
df = dd.read_csv('/home/py/lab1/lab1/datasets/optionstats/*.csv',parse_dates=True) #insertar la ruta de su carpeta con los CSV


time: 5.32 s


Ahora, vamos a asignar un indice al dataframe, en este caso queremos que sea la columna de la fecha, en este proceso, ya no puedo realziarlo de forma "lazy" asi que, le toca computar.

In [3]:
with ProgressBar(): # Para que nos muestre una barra de progreso.
    df = df.set_index('quotedate')

[########################################] | 100% Completed |  6min  8.0s
time: 6min 13s


El tiempo de carga, no es demasiado excedido, puedo aseguraros, que con pandas, el proceso tarda como unas 4 veces mas aproximadamente.

Ahora vamos a volver a utilizar el formato lazy del dataframe, asignamos las columnas que queremos tener en nuestro dataframe, y como vemos, es un proceso casi instantaneo.

Vamos a ver que columnas, tienen los archivos csv, y cuales vamos a seleccionar

In [24]:
df.columns

Index(['symbol', 'iv30call', 'iv30put', 'iv30mean', 'callvol', 'putvol',
       'totalvol', 'calloi', 'putoi'],
      dtype='object')

time: 5.65 ms


In [4]:
cols = ['symbol','iv30call','iv30put','iv30mean','callvol','putvol','totalvol','calloi', 'putoi']
df = df[cols]

time: 5.17 ms


Vale, que bonito todo, pero a mi me gusta pandas, y veo que estamos trabajando con un dask.dataframe, no con un pandas datraframe, Esto tiene solucion?

In [10]:
type(df)

dask.dataframe.core.DataFrame

time: 6.6 ms


Es muy simple, aqui es donde realmente se realiza todo el proceso, simplemente tenemos que pasar nuestro lazy dataframe a pandas mediante compute().

In [5]:
with ProgressBar():
    dfcomp = df.compute()

[########################################] | 100% Completed | 11min 27.4s
time: 11min 35s


In [9]:
type(dfcomp)

pandas.core.frame.DataFrame

time: 6.94 ms


In [11]:
dfcomp.head()

,symbol,iv30call,iv30put,iv30mean,callvol,putvol,totalvol,calloi,putoi
quotedate,,,,,,,,,
1/10/2003,A,0.7304,0.9472,0.8388,1020,5985,7005,55877,52091
1/10/2003,AA,0.5049,0.4441,0.4745,767,750,1517,54286,44490
1/10/2003,AAI,0.7767,0.7680,0.7724,0,100,100,3562,9546
1/10/2003,AAP,0.4373,0.4469,0.4421,40,50,90,7236,3035
1/10/2003,AAPL,0.9099,0.7751,0.8425,4099,799,4898,159958,120683


time: 29.8 ms


Y para finalizar este lab vamos a comprobar el ratio de perdida de nuestro dataframe, deberia ser 0, si algun valor no es igual a 0, deberiamos revisar nuestros datos y entender por donde se escapan los datos.

In [21]:
valores_null = df.isnull().sum() #sumamos los valores null
ratio_perdida = ((valores_null/df.index.size)*100) # sacamos el ratio entre valores null entre la longitud del dataframe

time: 1.05 s


Y como paso final, realizamos el .compute() para obtener nuestra serie de pandas.

In [19]:
with ProgressBar():
    ratio_perdida = ratio_perdida.compute()

[########################################] | 100% Completed | 11min 51.5s
time: 11min 58s


In [20]:
ratio_perdida

symbol      0.0
iv30call    0.0
iv30put     0.0
iv30mean    0.0
callvol     0.0
putvol      0.0
totalvol    0.0
calloi      0.0
putoi       0.0
dtype: float64

time: 6.96 ms
